In [4]:
from osgeo import gdal, ogr
import pandas as pd
import numpy as np
import os
import shapely as shply
import matplotlib.pyplot as plt

In [5]:
path = 'D:\\yyx\\GEE\\data\\NE_shp_2\\'
empty_list = []
for i in range(115, 137):
    for j in range(54, 38, -1):
        for year in range(1988, 2020):
            name = str(year) + '_' + str(i) + '_' + str(j) + '.shp'
            if not os.path.exists(path + name):
                empty_list.append(name)

In [6]:
pd.DataFrame(empty_list).to_csv('empty_list.csv')

In [2]:
path_shp = 'D:\\yyx\\GEE\\data\\test\\2014_126_54.shp'
start_year = 1988
end_year = 2019
# path_tif = 'D:\\yyx\\GEE\\data\\Buened_Arae_2003\\CCI_BA_2003_50N_120E.tif'

In [3]:
driver = ogr.GetDriverByName("ESRI Shapefile")
shp_test = driver.Open(path_shp, 1)
# tif_test = gdal.Open(path_tif)

In [4]:
layer = shp_test.GetLayer()

In [ ]:
def rename(layer, feature, year, year0):
    try:
        if feature[str(year - year0) + 'NDVI']:
            new_field = ogr.FieldDefn("NDVI" + str(i), ogr.OFTReal)
            layer.CreateField(new_field)
    except KeyError:
        print('skip')

In [ ]:
def pre_interfere(layer, feature, year0):
    if year0 - 2 <= start_year - 1:
        for year in range(start_year, year0 + 1):
            rename(layer, feature, year, year0)
    else:
        for year in range(year - 2, year0 + 1):
            rename(layer, feature, year, year0)


def aft_interfere(layer, feature, year0):
    if year0 + 5 >= end_year + 1:
        for year in range(year0, end_year + 1):
            rename(layer, feature, year, year0)
    else:
        for year in range(year0, year + 6):
            rename(layer, feature, year, year0)

In [26]:
new_field_fire = ogr.FieldDefn("fire", ogr.OFTReal)
new_field_rate = ogr.FieldDefn("rate", ogr.OFTReal)
layer.CreateField(new_field_fire)
layer.CreateField(new_field_rate)
for feature in layer:
    rate = feature['zonalsum'] * 69 / feature['count']
    if rate > 0.5:
        feature.SetField("fire", 1)
    else:
        feature.SetField("fire", 0)
    feature.SetField("rate", rate)
    layer.SetFeature(feature)